<a href="https://colab.research.google.com/github/reznimat/ppzd_sp/blob/main/semestralni_prace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Načtení knihoven**

In [13]:
import csv
import pandas as pd
import math as mt

**Načtení datových souborů**

In [55]:
# POZOR, pouze vzorek dat
#url_sales = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/SAMPLE_Online_Sales.csv'
url_sales = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Online_Sales.csv'
df_sales = pd.read_csv(url_sales)

url_coupons = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Discount_Coupon.csv'
df_coupons = pd.read_csv(url_coupons,sep=";")

print('**Sales**')
print(df_sales.count())
print('\n------\n')
print('**Coupons**')
print(df_coupons.count())


**Sales**
CustomerID             52924
Transaction_ID         52924
Transaction_Date       52924
Product_SKU            52924
Product_Description    52924
Product_Category       52924
Quantity               52924
Avg_Price              52924
Delivery_Charges       52924
Coupon_Status          52924
dtype: int64

------

**Coupons**
Month               204
Product_Category    204
Coupon_Code         204
Discount_pct        204
dtype: int64


**Obohacení df_sales o atribut Month (Date Abrv)**

In [56]:
# Obohacení df_sales o Month pole
for i, row in df_sales.iterrows():
  Month = pd.to_datetime(row.Transaction_Date).strftime("%b")
  df_sales["Month"] = Month

print('**Sales**')
print(df_sales.count())

**Sales**
CustomerID             52924
Transaction_ID         52924
Transaction_Date       52924
Product_SKU            52924
Product_Description    52924
Product_Category       52924
Quantity               52924
Avg_Price              52924
Delivery_Charges       52924
Coupon_Status          52924
Month                  52924
dtype: int64


**Spojení sales a coupons tabulek přes Product_Category a Month**

In [57]:
#Merge Sales a Coupons na základě Product Category a Month
df_sales_coupons = pd.merge(df_sales,df_coupons, on=["Product_Category","Month"])

print(df_sales_coupons.count())

CustomerID             52524
Transaction_ID         52524
Transaction_Date       52524
Product_SKU            52524
Product_Description    52524
Product_Category       52524
Quantity               52524
Avg_Price              52524
Delivery_Charges       52524
Coupon_Status          52524
Month                  52524
Coupon_Code            52524
Discount_pct           52524
dtype: int64


**Počet a výpis jedinečných produktových kategorií**

In [58]:
# Počet a výpis distinktních produktových kategorií
uniqPC = set()
for i, row in df_sales.iterrows():
  uniqPC.add(row["Product_Category"])

print("Produktových kategorií: "+ str(len(uniqPC)) )  # vypíše počet produktových kategorií
print(uniqPC)  # vypíše seznam produktových kategorií

Produktových kategorií: 20
{'Android', 'Google', 'Office', 'Notebooks & Journals', 'Bottles', 'Drinkware', 'Bags', 'Nest-Canada', 'Fun', 'More Bags', 'Nest', 'Gift Cards', 'Lifestyle', 'Backpacks', 'Housewares', 'Apparel', 'Waze', 'Headgear', 'Nest-USA', 'Accessories'}


**Suma tržeb pro kategorii Apparel**

In [59]:
# Hint: InvoiceValue = (( QuantityAvg_price)(1-Dicount_pct)*(1+GST))+Delivery_Charges
# nejsou započítány náklady na dopravu a daně

ApparelSales = 0
AvgOrderPrice = 0

for i, row in df_sales_coupons.iterrows():
  if row.Product_Category == 'Apparel':
    if row.Coupon_Status =='Not Used':
      ApparelSales += (row.Quantity*row.Avg_Price) # Bez slevy
    else:
      ApparelSales += (row.Quantity*row.Avg_Price)*(1-(row.Discount_pct/100)) # Se slevou

print(ApparelSales)

442432.6169999865


**Tržby po kategoriích**

In [60]:
for kategorie in uniqPC:
  CategorySales = 0
  for i, row in df_sales_coupons.iterrows():
    if row.Product_Category == kategorie:
      if row.Coupon_Status =='Not Used':
        CategorySales += (row.Quantity*row.Avg_Price) # Bez slevy
      else:
        CategorySales += (row.Quantity*row.Avg_Price)*(1-(row.Discount_pct/100)) # Se slevou

  print(kategorie,CategorySales)

Android 549.561
Google 0
Office 204502.85999999836
Notebooks & Journals 82774.62299999982
Bottles 5115.91899999999
Drinkware 150824.03099999993
Bags 113498.33500000046
Nest-Canada 53117.49900000007
Fun 0
More Bags 0
Nest 387210.79399999086
Gift Cards 14585.134
Lifestyle 55484.46399999944
Backpacks 0
Housewares 3555.649000000001
Apparel 442432.6169999865
Waze 4736.465999999992
Headgear 38873.20000000003
Nest-USA 1905998.3870002662
Accessories 5539.064999999993
